### 결측값 확인하기 : isnull, notnull

In [ ]:
import pandas as pd
import numpy as np

df_left = pd.DataFrame({
    'KEY': ['K0', 'K1', 'K2', 'K3'],
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': [0.5, 2.2, 3.6, 0.4]})

df_right = pd.DataFrame({
    'KEY': ['K2', 'K3', 'K4', 'K5'],
    'C': ['C2', 'C3', 'C4', 'C5'],
    'D': ['D2', 'D3', 'D4', 'D5']})

df_all = pd.merge(df_left, df_right, how='outer', on='KEY')
df_all

#- DataFrame 전체의 결측값 여부 확인
#- df.isnull(), pd.isnull(df), df.notnull(), pd.notnull(df)
#- df.isnull()과 pd.isnull(df)은 같은 의미
#- df.notnull()과 pd.notnull(df)은 같은 의미

pd.isnull(df_all)
pd.notnull(df_all)

df_all.isnull()
df_all.notnull()

#- 특정 변수, 특정 컬럼에 결측값 입력하기 ** 중요!
#- ** string 형식인 경우에 None을 할당하면, 'None'으로 입력
#- ** float 형식인 경우에 None을 할당하면,  NaN으로 자동으로 입력됨
df_all.loc[[0,1], ['A', 'B']] = None
df_all

#- 컬럼별 결측값 개수 구하기 : df.isnull().sum()
df_all.isnull().sum()

#- 행단위로 결측값 개수 구하기
df_all.isnull().sum(1)

### 결측값 연산

In [ ]:
df = pd.DataFrame(
  np.arange(10).reshape(5, 2),
  columns= ['C1', "C2"],
  index = ['a', 'b', 'c', 'd', 'e'])

df.loc[['b', 'e'], ['C1']] = None
df.loc[['b', 'c'], ['C2']] = None

#- 결측 존재시, sum(), cumsum() --> 결측을 제외하고 계산
df.sum()
df['C1'].sum()
df['C1'].cumsum()

#- mean(), std() 연산 시 : NaN은 분석 대상에서 제외
#  --> 산술 평균이나 표준편차 값을 계산할 때, NaN이 포함된 경우는 아예 분모에서 카운트를 제외
df.mean()
df.std()

#- DataFrame 컬럼 간 연산 시, 하나의 Row에 NaN이 하나라도 포함되어 있으면 NaN return
df['C3'] = df['C1'] + df['C2']
df

#- DataFrame 끼리의 연산(df1 + df2) 
#- 같은 명의 컬럼인 경우에는 NaN을 0으로 계산하고 연산. 
#- 같은 명의 컬럼이 없는 경우는 모든 값을 NaN으로 연산
df2 = pd.DataFrame({
    'C1':[1, 1, 1, 1, 1],
    'C4':[1, 1, 1, 1, 1]},
    index = ['a', 'b', 'c', 'd','e'])

### 결측값 채우기
- 결측값을 특정 값으로 채우기
- 결측값을 앞 방향 또는 뒷 방향으로 채우기
- 결측값 채우는 회수를 제한하기
- 결측값을 특정 통계량으로 채우기
- 결측값을 다른 변수의 값으로 대체하기

In [ ]:
df = pd.DataFrame(
  np.arange(15).reshape(5, 3),
  columns= ['C1', "C2", "C3"])

#- 결측값으로 대체하는 방법은 None으로 할당하거나, np.nan 할당
df.iloc[0,0] = None
df.loc[1, ['C1', 'C3']] = np.nan
df.loc[2, ['C2']] = np.nan
df.loc[3, ['C2']] = np.nan
df.loc[4, ['C3']] = np.nan
df

#- (1) 결측값을 특정 값으로 채우기 : df.fillna()
df_fill_zero = df.fillna(0)
df_fill_zero

df_fill_missing = df.fillna("missing")
df_fill_missing

#- (2) 결측값을 앞 방향 또는 뒷 방향으로 채우기
df.fillna(method = 'ffill')   #- 앞 방향
df.fillna(method = 'pad')     #- 앞 방향

df.fillna(method = 'bfill')     #- 뒷 방향
df.fillna(method = 'backfill')  #- 뒷 방향

#- 앞/뒤 방향으로 결측값 채우는 회수를 제한하기
#- --> 컬럼별 데이터에서 결측값이 연속으로 생성되어 있는 부분에서 limit을 걸어서 NaN을 채울 최대 수를 지정
#- limit paramter로 값 지정

df.fillna(method = 'ffill', limit = 1)
df.fillna(method = 'bfill', limit = 1)

#- 결측값을 변수별 평균으로 대체하기 : df.where

#- 컬럼에 존재하는 결측값을 컬럼별로 할당
#- 하단의 두 가지 방법 사용 가능
df.fillna(df.mean())
df.where(df.notnull(), df.mean(), axis = 'columns')

#- C1 컬럼의 평균을 가지고, (C1, C2, C3) 결측값 대체 예제
df.fillna(df.mean()['C1'])

#- C1, C2 컬럼만 각각 결측값 대체하고 C3는 대체 안하는 경우
df.fillna(df.mean()['C1':'C2'])

#- 결측값을 다른 변수의 값으로 대체하기
#- C2 컬럼에 결측값이 없으면 C2 값을 그대로 사용, 있으면 C1 컬럼의 값을 가져다가 대체
#- 단순 for loop를 사용하는 것 보다, np.where, pd.notnull를 사용하는 것이 훨씬 유리
df['C2_New'] = np.where(pd.notnull(df['C2']) == True, df['C2'], df['C1'])

### 결측값 있는 행 제거 : dropna(axis = 0)

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(
    np.random.randn(5,4),
    columns = ['C1', 'C2', 'C3', 'C4'])
df.iloc[[0,1], [0]] = None
df.iloc[[2], [1]] = None
df_drop_row = df.dropna(axis = 0) # 행제거
df_drop_col = df.dropna(axis = 1) # 열제거
df['C1'].dropna() # C1 컬럼만 선택하여 열제거

### 결측값 보간하기(interporlation of missing values)

In [ ]:
# 시계열 데이터의 값에 선형으로 비례하는 방식으로 값을 보간(interpolate)
from datetime import datetime
datestrs = ['12/01/2016', '12/03/2016', '12/04/2016', '12/10/2016']
datestrs = pd.to_datetime(datestrs)
ts = pd.Series([1, np.nan, np.nan, 10], index = datestrs)

# (1) 시계열 데이터의 값에 선형으로 비례하는 방식으로 결측값 보간
ts.interpolate()
ts.interpolate(method='time')  # --> method = time 지정시, index 날짜를 기준으로 linear하게 보간됨

# (2) DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간
df = pd.DataFrame({'C1':[1, 2, np.nan, np.nan, 5], 'C2':[6, 8, 10, np.nan, 20]})
df.interpolate(method = 'values')

# (3) 결측값 보간 개수 제한하기 : limit
ts.interpolate(method = 'time', limit = 1)
df.interpolate(method = 'values', limit = 1)

# (4) 보간 방향 설정하기 : limit_direction = both, forward, backward
ts.interpolate(method = 'time', limit = 1, limit_direction = 'both')
df.interpolate(method = 'values', limit = 1, limit_direction = 'backward')

In [ ]:
from datetime import datetime
datestrs = ['12/01/2016', '12/03/2016', '12/04/2016', '12/10/2016']
datestrs = pd.to_datetime(datestrs)
ts = pd.Series([1, np.nan, np.nan, 10], index = datestrs)

df = pd.DataFrame({'C1':[1, 2, np.nan, np.nan, 5], 'C2':[6, 8, 10, np.nan, 20]})


### 결측값, 원래 값을 다른 값으로 대체하기 : replace

In [ ]:
## replace 와 fillna 함수는 유사한 면이 있지만,
## replace 함수는 결측값이 아닌 대체값 용도로 사용이 가능하며, list, mapping dict 등으로 좀더 유연하고 포괄적으로 사용할 수 있는 장점이 있음
import pandas as pd
import numpy as np
ser = pd.Series([1, 2, 3, np.nan])

# (1) 결측값, 실측값으로 대체
ser.replace(2, 20)
ser.replace(np.nan, 20)

# (2) list를 list로 대체 
ser.replace([1, 2, 3, 4, np.nan], [6, 7, 8, 9, 10])

# (3) mapping dict로 원래 값, 교체할 값 매핑 : replace({old1 : new1, old2 : new2})
ser.replace({1:100, 2:200, 3:300, np.nan:400})

# (4) DataFrame의 특정 컬럼 값 교체하기: df.replace({'col1':old_val}, {'col1':new_val})
df = pd.DataFrame(
    {'C1':['a_old', 'b', 'c', 'd', 'e'],
     'C2':[1,2,3,4,5],
     'C3':[6,7,8,9,np.nan]}
)
df = df.replace({'C1':'a_old'}, {'C1':'a'})
df = df.replace({'C3':np.nan}, {'C3': 10})
df

### 중복값 확인 및 처리 

In [ ]:
import pandas as pd
df = pd.DataFrame(
    {'key1':['a', 'b', 'b', 'c', 'c'],
     'key2':['v', 'w', 'w', 'x', 'y'],
     'col':[1, 2, 3, 4, 5]}
)

# (1) 중복 데이터가 있는지 확인 : df.duplicated()
df.duplicated(['key1'])
df.duplicated(['key1', 'key2'])

# (2) 중복이 있으면 처음과 끝 중 무슨 값을 남길 것인가? : keep = 'first', 'last', False
# keep = 'first' --> 중복시 첫번째 값만 False, 나머지는 True
# keep = 'last'  --> 중복시 마지막 값만 False, 나머지는 True
# keep = False   --> 중복값 모두 True, 중복되는 행 자체를 남기지 않음
df.duplicated(['key1'], keep = 'first')
df.duplicated(['key1'], keep = 'last')
df.duplicated(['key1'], keep = False)

# (3) drop_duplicates()
df.drop_duplicates(['key1'], keep = 'first')
df.drop_duplicates(['key1'], keep = 'last')
df.drop_duplicates(['key1'], keep =  False)

0    False
1     True
2     True
3     True
4     True
dtype: bool